# Root cause analysis of failure using machine learning


Root cause analysis = process of identifying errors and flaws in the quality of manufactured product

ML: Graph and bayesian networks - pgmpy





---
### 1. Loading required libraries and data

In [6]:
import pandas as pd

data=pd.read_csv('../data/WIDS Dataset_2020_Adj.csv')

---
### 2. Exploring data
- renaming and dropping columns
- dropping duplicates

In [22]:
data.shape

(75750, 53)

In [8]:
data.head()

,X,Date,Zone1Position,Zone2Position,Zone3Position,SKU,Zone1_Row_Num,Zone1_Col_Num,Zone2_Row_Num,Zone2_Col_num,...,Zone2_Humidity_Max,Zone2_Humidity_Range,Zone3_Humidity_Avg,Zone3_Humidity_Min,Zone3_Humidity_Max,Zone3_Humidity_Range,Block_Num,Block_Position,Block_Orientation,Result_Type
0,1,09/01/2020,3.0,2.0,3.0,A001,1.0,3.0,1.0,2.0,...,61.80,10.0,39.19,33.19,45.19,12.0,1000.0,1.0,1,Defect_1
1,2,03/12/2019,8.0,4.0,3.0,A001,2.0,4.0,2.0,2.0,...,58.12,16.0,48.50,40.50,56.50,16.0,1003.0,1.0,1,Defect_1
2,3,21/11/2019,4.0,2.0,3.0,B003,1.0,4.0,1.0,2.0,...,50.97,10.0,52.20,46.20,58.20,12.0,1001.0,4.0,1,PASS
3,4,22/11/2019,6.0,3.0,1.0,B003,2.0,2.0,2.0,1.0,...,59.13,4.0,39.29,31.29,47.29,16.0,1003.0,2.0,1,PASS
4,5,23/12/2019,3.0,2.0,3.0,B003,1.0,3.0,1.0,2.0,...,65.80,18.0,45.37,42.37,48.37,6.0,1002.0,1.0,1,PASS


> Dropping `X` column as it's unnecessary

In [26]:
data.drop(columns=['X'], inplace=True)

In [27]:
data.duplicated().value_counts()

False    75000
True       750
Name: count, dtype: int64

> Dropping the `750` duplicated rows

In [29]:
data.drop_duplicates(inplace=True)

In [37]:
# Checking column names
data.columns.tolist()

['Date',
 'Zone1Position',
 'Zone2Position',
 'Zone3Position',
 'SKU',
 'Zone1_Row_Num',
 'Zone1_Col_Num',
 'Zone2_Row_Num',
 'Zone2_Col_num',
 'Zone3_Row_Num',
 'Zone3_Col_Num',
 'Zone1_Left_Block_Bin',
 'Zone1_Right_Block_Bin',
 'Zone1_Area',
 'Zone3_Area',
 'Zone1_Dur',
 'Zone2_Dur',
 'Zone3_Dur',
 'Zone1_Out_Zone2_In_Dur',
 'Zone1_Out_Zone3_In_Dur',
 'Zone2_Out_Zone3_In_Dur',
 'Zone1_In_Zone3_Out_Dur',
 'Zone1_In_Zone2_Out_Dur',
 'Zone2_In_Zone3_Out_Dur',
 'Zone1_Temp_Avg',
 'Zone1_Temp_Min',
 'Zone1_Temp_Max',
 'Zone1_Temp_Range',
 'Zone2_Temp_Avg',
 'Zone2_Temp_Min',
 'Zone2_Temp_Max',
 'Zone2_Temp_Range',
 'Zone3_Temp_Avg',
 'Zone3_Temp_Min',
 'Zone3_Temp_Max',
 'Zone3_Temp_Range',
 'Zone1_Humidity_Avg',
 'Zone1_Humidity_Min',
 'Zone1_Humidity_Max',
 'Zone1_Humidity_Range',
 'Zone2_Humidity_Avg',
 'Zone2_Humidity_Min',
 'Zone2_Humidity_Max',
 'Zone2_Humidity_Range',
 'Zone3_Humidity_Avg',
 'Zone3_Humidity_Min',
 'Zone3_Humidity_Max',
 'Zone3_Humidity_Range',
 'Block_Num',
 'Bloc

In [38]:
# I will capitalize the `num` in `Zone2_Col_num` for consistency purposes
data.rename(columns={'Zone2_Col_num':'Zone2_Col_Num'}, inplace=True)

In [39]:
data.columns.tolist()

['Date',
 'Zone1Position',
 'Zone2Position',
 'Zone3Position',
 'SKU',
 'Zone1_Row_Num',
 'Zone1_Col_Num',
 'Zone2_Row_Num',
 'Zone2_Col_Num',
 'Zone3_Row_Num',
 'Zone3_Col_Num',
 'Zone1_Left_Block_Bin',
 'Zone1_Right_Block_Bin',
 'Zone1_Area',
 'Zone3_Area',
 'Zone1_Dur',
 'Zone2_Dur',
 'Zone3_Dur',
 'Zone1_Out_Zone2_In_Dur',
 'Zone1_Out_Zone3_In_Dur',
 'Zone2_Out_Zone3_In_Dur',
 'Zone1_In_Zone3_Out_Dur',
 'Zone1_In_Zone2_Out_Dur',
 'Zone2_In_Zone3_Out_Dur',
 'Zone1_Temp_Avg',
 'Zone1_Temp_Min',
 'Zone1_Temp_Max',
 'Zone1_Temp_Range',
 'Zone2_Temp_Avg',
 'Zone2_Temp_Min',
 'Zone2_Temp_Max',
 'Zone2_Temp_Range',
 'Zone3_Temp_Avg',
 'Zone3_Temp_Min',
 'Zone3_Temp_Max',
 'Zone3_Temp_Range',
 'Zone1_Humidity_Avg',
 'Zone1_Humidity_Min',
 'Zone1_Humidity_Max',
 'Zone1_Humidity_Range',
 'Zone2_Humidity_Avg',
 'Zone2_Humidity_Min',
 'Zone2_Humidity_Max',
 'Zone2_Humidity_Range',
 'Zone3_Humidity_Avg',
 'Zone3_Humidity_Min',
 'Zone3_Humidity_Max',
 'Zone3_Humidity_Range',
 'Block_Num',
 'Bloc

In [30]:
data.duplicated().value_counts()

False    75000
Name: count, dtype: int64

### 3. Handling missing values

In [40]:
data.isnull().sum()

Date                      42
Zone1Position             36
Zone2Position             43
Zone3Position             37
SKU                       38
Zone1_Row_Num             34
Zone1_Col_Num             42
Zone2_Row_Num             46
Zone2_Col_Num             34
Zone3_Row_Num             37
Zone3_Col_Num             35
Zone1_Left_Block_Bin      42
Zone1_Right_Block_Bin     40
Zone1_Area                41
Zone3_Area                32
Zone1_Dur                 37
Zone2_Dur                 41
Zone3_Dur                 46
Zone1_Out_Zone2_In_Dur    42
Zone1_Out_Zone3_In_Dur    41
Zone2_Out_Zone3_In_Dur    41
Zone1_In_Zone3_Out_Dur    34
Zone1_In_Zone2_Out_Dur    44
Zone2_In_Zone3_Out_Dur    38
Zone1_Temp_Avg            52
Zone1_Temp_Min            46
Zone1_Temp_Max            38
Zone1_Temp_Range          40
Zone2_Temp_Avg            51
Zone2_Temp_Min            51
Zone2_Temp_Max            37
Zone2_Temp_Range          37
Zone3_Temp_Avg            43
Zone3_Temp_Min            26
Zone3_Temp_Max

<center> <h2>Production process</h2> </center>

<img src="../img/prod.png" width="60%" height="60%"/>
<img src="../img/z1-z2.png" width="45%" height="45%"/>
<img src="../img/z2-z3.png" width="60%" height="60%"/>


> **Product trip schema**: Z1 > Z2 >Z3

> Z1 1,2 > Z2 1 > Z3 left 1,2,4 <br>
Z1 3,4 > Z2 2 > Z3 right 2,3,6 <br>
Z1 5,6 > Z2 3 > Z3 right 1,4,5 <br>
Z1 7,8 > Z2 4 > Z3 left 3,5,6 <br>

> **Deduction process:** <br>
Position = (row-1)* counts in row +col <br>
Column = position - (row-1)* counts in row  <br>
Row =((position - column)/counts in row)+1 <br>

#### A. Inputting missing values in the position, column and row columns

In [42]:
zones={'Zone1':4, 'Zone2':2, 'Zone3':3}

for key, value in zones.items():
    data[key+"Position"].fillna(value=(((data[key+"_Row_Num"] -1) * value) + data[key+"_Col_Num"]), inplace=True)
    data[key+"_Col_Num"].fillna(value=(data[key+"Position"] -((data[key+"_Row_Num"] -1) * value)), inplace=True)
    data[key+"_Row_Num"].fillna(value=(((data[key+"Position"]-data[key+"_Col_Num"])/value)+1), inplace=True)

#### B. Inputting missing values in block bins and areas

In [86]:
zone1_l_r={'Top Left':[[1,2],[1,0]],'Top Right':[[3,4],[0,1]],'Bottom Left':[[5,6],[1,0]],
           'Bottom Right':[[7,8],[0,1]]}

for key,values in zone1_l_r.items():
    for value in values[0]:
        indices= data['Zone1Position'] == value
        data.loc[indices,'Zone1_Left_Block_Bin']=data.loc[indices,'Zone1_Left_Block_Bin'].fillna(value=values[1][0])
        data.loc[indices,'Zone1_Right_Block_Bin']=data.loc[indices,'Zone1_Right_Block_Bin'].fillna(value=values[1][1])
        data.loc[indices,'Zone1_Area']=data.loc[indices,'Zone1_Area'].fillna(value=key)

In [90]:
zone3_area={'Top Left':[1,2,4],'Bottom Right':[3,5,6]}
for key,values in zone3_area.items():
    for value in values:
        indices= data['Zone3Position'] == value
        data.loc[indices,'Zone3_Area']=data.loc[indices,'Zone3_Area'].fillna(value=key)

#### C. Inputting missing values for temperature and humidity columns: min, max range

In [94]:
for key in zones.keys():
    data[key+"_Temp_Min"].fillna(value=data[key+"_Temp_Max"]-data[key+"_Temp_Range"], inplace=True)
    data[key+"_Temp_Max"].fillna(value=data[key+"_Temp_Range"]+data[key+"_Temp_Min"], inplace=True)
    data[key+"_Temp_Range"].fillna(value=data[key+"_Temp_Max"]-data[key+"_Temp_Min"], inplace=True)
    data[key+"_Humidity_Min"].fillna(value=data[key+"_Humidity_Max"]-data[key+"_Humidity_Range"], inplace=True)
    data[key+"_Humidity_Max"].fillna(value=data[key+"_Humidity_Range"]+data[key+"_Humidity_Min"], inplace=True)
    data[key+"_Humidity_Range"].fillna(value=data[key+"_Humidity_Max"]-data[key+"_Humidity_Min"], inplace=True)


In [95]:
data.isnull().sum()

Date                      42
Zone1Position              0
Zone2Position              0
Zone3Position              0
SKU                       38
Zone1_Row_Num              0
Zone1_Col_Num              0
Zone2_Row_Num              0
Zone2_Col_Num              0
Zone3_Row_Num              0
Zone3_Col_Num              0
Zone1_Left_Block_Bin       0
Zone1_Right_Block_Bin      0
Zone1_Area                 0
Zone3_Area                 0
Zone1_Dur                 37
Zone2_Dur                 41
Zone3_Dur                 46
Zone1_Out_Zone2_In_Dur    42
Zone1_Out_Zone3_In_Dur    41
Zone2_Out_Zone3_In_Dur    41
Zone1_In_Zone3_Out_Dur    34
Zone1_In_Zone2_Out_Dur    44
Zone2_In_Zone3_Out_Dur    38
Zone1_Temp_Avg            52
Zone1_Temp_Min             0
Zone1_Temp_Max             0
Zone1_Temp_Range           0
Zone2_Temp_Avg            51
Zone2_Temp_Min             0
Zone2_Temp_Max             0
Zone2_Temp_Range           0
Zone3_Temp_Avg            43
Zone3_Temp_Min             0
Zone3_Temp_Max

In [96]:
data[['Zone1_Dur','Zone2_Dur','Zone3_Dur','Zone1_Out_Zone2_In_Dur','Zone1_Out_Zone3_In_Dur','Zone2_Out_Zone3_In_Dur',
      'Zone1_In_Zone3_Out_Dur','Zone1_In_Zone2_Out_Dur','Zone2_In_Zone3_Out_Dur']]

,Zone1_Dur,Zone2_Dur,Zone3_Dur,Zone1_Out_Zone2_In_Dur,Zone1_Out_Zone3_In_Dur,Zone2_Out_Zone3_In_Dur,Zone1_In_Zone3_Out_Dur,Zone1_In_Zone2_Out_Dur,Zone2_In_Zone3_Out_Dur
0,14.79,49.55,26.0,4.79,62.17,6.71,101.73,66.90,81.67
1,17.19,47.53,29.0,3.04,56.72,3.34,99.31,67.68,79.16
2,16.54,50.23,23.0,4.79,60.24,7.30,100.02,72.82,80.18
3,8.64,49.41,28.0,3.66,58.16,6.69,100.45,67.54,80.45
4,17.19,49.66,30.0,3.04,61.47,5.56,99.74,69.20,79.60
...,...,...,...,...,...,...,...,...,...
74995,13.05,51.11,24.0,4.36,61.00,7.18,100.18,67.54,78.09
74996,15.28,51.72,24.0,4.78,64.38,7.22,100.13,69.72,77.17
74997,15.28,49.93,28.0,4.05,64.38,3.34,98.64,72.57,78.13
74998,11.90,49.13,36.0,5.14,60.90,3.34,100.33,71.94,82.35
